In [ ]:
!pip install floweaver

In [ ]:
!pip install ipysankeywidget


     |████████████████████████████████| 1.2 MB 5.3 MB/s 
Traceback (most recent call last):
  File "/usr/local/bin/jupyter-nbextension", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py", line 267, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/notebook/nbextensions.py", line 988, in start
    super(NBExtensionApp, self).start()
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py", line 256, in start
    self.subapp.start()
  File "/usr/local/lib/python2.7/dist-packages/notebook/nbextensions.py", line 896, in start
    self.toggle_nbextension_python(self.extra_args[0])
  File "/usr/local/lib/python2.7/dist-packages/notebook/nbextensions.py", line 872, in toggle_nbextension_python
    l

In [ ]:
!jupyter nbextension enable --py --sys-prefix ipysankeywidget

Traceback (most recent call last):
  File "/usr/local/bin/jupyter-nbextension", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py", line 267, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/notebook/nbextensions.py", line 988, in start
    super(NBExtensionApp, self).start()
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py", line 256, in start
    self.subapp.start()
  File "/usr/local/lib/python2.7/dist-packages/notebook/nbextensions.py", line 896, in start
    self.toggle_nbextension_python(self.extra_args[0])
  File "/usr/local/lib/python2.7/dist-packages/notebook/nbextensions.py", line 872, in toggle_nbextension_python
    logger=self.log)
  File "/usr/local/lib/python2.7/dist-pac

In [ ]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [ ]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/gsdavis1959/Data/master/WorldCruiseData.csv")
df

,ROUTE,EMBARK_PORT,DISEMBARK_PORT,CABIN_TYPE,DURATION,CRUISE_REGION,BOOKING_WEEK_PRIOR,CANCEL_WEEK_PRIOR,EMBARK_REGION,DEBARK_REGION,PRICE_PAID,NO_OF_PEOPLE_ON_BOOKING
0,SOU-AKL,SOU,AKL,O,40,West Bound,33,33.0,UK,New Zealand,2811.127882,2
1,SOU-AKL,SOU,AKL,O,40,West Bound,45,NaN,UK,New Zealand,5171.396346,2
2,SOU-AKL,SOU,AKL,B,40,West Bound,42,NaN,UK,New Zealand,6343.673826,2
3,SOU-BNE,SOU,BNE,B,47,West Bound,36,35.0,UK,Australia - Queensland,9784.513217,2
4,SOU-SFO,SOU,SFO,B,24,Panama Canal,57,NaN,UK,USA - West Coast,4182.892388,2
...,...,...,...,...,...,...,...,...,...,...,...,...
3918,HKG-SOU,HKG,SOU,B,36,Southeast Asia,42,41.0,China - Southern,UK,5084.221402,2
3919,DXB-SOU,DXB,SOU,S,19,Middle East,47,NaN,UAE,UK,5636.195082,2
3920,DXB-SOU,DXB,SOU,B,19,Middle East,24,NaN,UAE,UK,1596.550657,2
3921,DXB-SOU,DXB,SOU,B,19,Middle East,12,12.0,UAE,UK,2160.742284,2


In [ ]:
from floweaver import *

flows = (
    df.groupby(["EMBARK_PORT", "DISEMBARK_PORT", "CRUISE_REGION"])
    .agg({"PRICE_PAID": "mean"})
    .dropna()
    .reset_index()
)

In [ ]:
flows = (
    flows.rename(
        columns={
            "EMBARK_PORT": "source",
            "DISEMBARK_PORT": "target",
            "CRUISE_REGION": "type",
            "PRICE_PAID": "value",
        }
    )
)

In [ ]:
from floweaver import *

nodes = {
    "embark_port": ProcessGroup(flows["source"].unique().tolist()),
    "disembark_port": ProcessGroup(flows["target"].unique().tolist()),
}

In [ ]:
ordering = [["embark_port"], ["disembark_port"]]

In [ ]:
bundles = [Bundle("embark_port", "disembark_port")]

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
sdd = SankeyDefinition(nodes, bundles, ordering)
weave(sdd, flows).to_widget()

SankeyWidget(layout=Layout(height='500', width='700'), links=[{'source': 'embark_port^*', 'target': 'disembark…

In [ ]:
# Create partitions for each type of port
embark_port = Partition.Simple("process", flows["source"].unique().tolist())
disembark_port = Partition.Simple("process", flows["target"].unique().tolist())

nodes["embark_port"].partition = embark_port
nodes["disembark_port"].partition = disembark_port

# Create Sankey diagram
weave(sdd, flows).to_widget()

SankeyWidget(groups=[{'id': 'embark_port', 'type': 'process', 'title': '', 'nodes': ['embark_port^AKL', 'embar…

In [ ]:
# specify groups 
continents = [
    ("Oceania", ["AKL"]),
    ("Australia", ["BNE", "SYD"]),
    ("North America", ["SFO"]),
    ("Europe", ["SOU"]),
    ("Asia", ["HKG", "SIN", "DXB"]),
]

continent_groups = Partition.Simple("process", continents)

# redefine partition
nodes["embark_port"].partition = continent_groups
nodes["disembark_port"].partition = continent_groups

# create a Sankey diagram
weave(sdd, flows).to_widget()

SankeyWidget(groups=[{'id': 'embark_port', 'type': 'process', 'title': '', 'nodes': ['embark_port^Oceania', 'e…

In [ ]:
weave(sdd, flows, link_color=QuantitativeScale("value")).to_widget()

SankeyWidget(groups=[{'id': 'embark_port', 'type': 'process', 'title': '', 'nodes': ['embark_port^Oceania', 'e…

In [ ]:
weave(
    sdd, flows, link_color=QuantitativeScale("value", palette="Blues_9")
).to_widget()

SankeyWidget(groups=[{'id': 'embark_port', 'type': 'process', 'title': '', 'nodes': ['embark_port^Oceania', 'e…

In [ ]:
cruise_region = Partition.Simple("type", flows.type.unique())

sdd = SankeyDefinition(
    nodes,
    bundles,
    ordering,
    flow_partition=cruise_region,
)
weave(sdd, flows).to_widget()

SankeyWidget(groups=[{'id': 'embark_port', 'type': 'process', 'title': '', 'nodes': ['embark_port^Oceania', 'e…

In [ ]:
# 1. Define a new waypoint node
cruise_region = Partition.Simple("type", flows.type.unique())

nodes["waypoint"] = Waypoint(cruise_region)

# 2. Update the ordering to show where the waypoint goes: in the middle
ordering = [
    ["embark_port"],
    ["waypoint"],
    ["disembark_port"],
]

# 3. Update the bundle definition to send the flows via the waypoint
bundles = [
    Bundle("embark_port", "disembark_port", waypoints=["waypoint"]),
]

# Update the SDD with the new nodes, ordering & bundles.
sdd = SankeyDefinition(nodes, bundles, ordering)

weave(sdd, flows, link_color=QuantitativeScale("value", palette='Blues_9')).to_widget()

SankeyWidget(groups=[{'id': 'embark_port', 'type': 'process', 'title': '', 'nodes': ['embark_port^Oceania', 'e…

In [1]:
# Example 2
import pandas as pd

data = pd.read_csv('https://gist.githubusercontent.com/nicolasesnis/eb3b35545e97926ab53e0617c5e4b639/raw/824eddae70819003214b5d35a83d8cb885852666/raw.csv')[
    ['user_id', 'time_install', 'event_name', 'time_event']]

# Start with making sure that time_event and time_insrall are Pandas Datetime types:
data['time_event'] = pd.to_datetime(data['time_event'], unit='s') # Unit = "s" is required here since the raw data format is unique timestamp. Remove this param if your data is already a datetime like data.
data['time_install'] = pd.to_datetime(data['time_install'], unit='s')

# Make sure that there's no event occurring before time_install
data = data[data.time_event >= data.time_install]
data.head()

,user_id,time_install,event_name,time_event
0,4c6065c9466bc68e324e316edfb0227ff7cccc6c,1970-01-01 12:11:32.978136574,purchase,1970-01-01 12:11:32.983807870
1,f3049eac4788ffd4482390f8333d7e1adbf4c5a1,1970-01-01 12:11:36.220370370,signup,1970-01-01 12:11:36.220798611
2,f3049eac4788ffd4482390f8333d7e1adbf4c5a1,1970-01-01 12:11:36.220370370,purchase,1970-01-01 12:11:36.221886574
4,723fba1295b9a7c8321bbc433f87629b90660582,1970-01-01 12:11:32.000046296,reopen,1970-01-01 12:11:33.577488426
5,723fba1295b9a7c8321bbc433f87629b90660582,1970-01-01 12:11:32.000046296,signup,1970-01-01 12:11:32.002523148


In [2]:
# The initial data Pandas DataFrame must have these 4 columns:
# user_id | time_install | event_name | time_event
# - user_id (string): the unique identifier of a user
# - time_install (Pandas datetime): the time when the user installed the app (there should be 1 time_install per user_id)
# - event_name (string): the name of a specific in-app event (there can be many event_name per user_id)
# - time_event (Pandas datetime): the time of each event (there should be 1 time_event per user_id)

# Edit this dataframe so that installs are passed as events

# Create a new DF from the data DF containing only install data
installs = data[['user_id', 'time_install']].sort_values(
    'time_install').drop_duplicates('user_id')

# Create an install column containing dummy "install" events
installs['event_name'] = 'install'

# Create an event_type column to keep the information of install vs other events
installs['event_type'] = 'install'

# Rename time_install to time_event

installs.rename(columns={'time_install': 'time_event'}, inplace=True)

# In the data DF, keep only events data and create the event_type column
data = data[['user_id', 'event_name',
             'time_event']].drop_duplicates()
data['event_type'] = 'in_app_action'

In [3]:
# Concatenate the two DataFrames
data = pd.concat([data, installs[data.columns]])

# Based on the time of events, we can compute the rank of each action at the user_id level:

# a) Sort ascendingly per user_id and time_event
# sort by event_type to make sure installs come first
data.sort_values(['user_id', 'event_type', 'time_event'],
                 ascending=[True, False, True], inplace=True)

In [4]:
# b) Group by user_id
grouped = data.groupby('user_id')

# c) Define a ranking function based on time_event, using the method = 'first' param to ensure no events have the same rank

def rank(x): return x['time_event'].rank(method='first').astype(int)

# d) Apply the ranking function to the data DF into a new "rank_event" column
data["rank_event"] = grouped.apply(rank).reset_index(0, drop=True)

# Add, each row, the information about the next_event

# a) Regroup by user_id
grouped = data.groupby('user_id')

# b) The shift function allows to access the next row's data. Here, we'll want the event name

def get_next_event(x): return x['event_name'].shift(-1)

# c) Apply the function into a new "next_event" column
data["next_event"] = grouped.apply(
    lambda x: get_next_event(x)).reset_index(0, drop=True)

# Likewise, we can compute time from each event to its next event:

# a) Regroup by user_id 
grouped = data.groupby('user_id')

# b) We make use one more time of the shift function:

def get_time_diff(
    x): return x['time_event'].shift(-1) - x['time_event']

# c) Apply the function to the data DF into a new "time_to_next" column
data["time_to_next"] = grouped.apply(
    lambda x: get_time_diff(x)).reset_index(0, drop=True)

# Here we'll plot the journey up to the 10th action. This can be achieved by filtering the dataframe based on the rank_event column that we computed:
data = data[data.rank_event < 10]

# Check that you have only installs at rank 1:
data[data['rank_event'] == 1].event_name.unique()

array(['install'], dtype=object)

In [5]:
import seaborn as sns

# Working on the nodes_dict

all_events = list(data.event_name.unique())

# Create a set of colors that you'd like to use in your plot.
palette = ['50BE97', 'E4655C', 'FCC865',
           'BFD6DE', '3E5066', '353A3E', 'E6E6E6']
#  Here, I passed the colors as HEX, but we need to pass it as RGB. This loop will convert from HEX to RGB:
for i, col in enumerate(palette):
    palette[i] = tuple(int(col[i:i+2], 16) for i in (0, 2, 4))

# Append a Seaborn complementary palette to your palette in case you did not provide enough colors to style every event
complementary_palette = sns.color_palette(
    "deep", len(all_events) - len(palette))
if len(complementary_palette) > 0:
    palette.extend(complementary_palette)

output = dict()
output.update({'nodes_dict': dict()})

i = 0
for rank_event in data.rank_event.unique(): # For each rank of event...
    # Create a new key equal to the rank...
    output['nodes_dict'].update(
        {rank_event: dict()}
    )
    
    # Look at all the events that were done at this step of the funnel...
    all_events_at_this_rank = data[data.rank_event ==
                                   rank_event].event_name.unique()
    
    # Read the colors for these events and store them in a list...
    rank_palette = []
    for event in all_events_at_this_rank:
        rank_palette.append(palette[list(all_events).index(event)])
    
    # Keep trace of the events' names, colors and indices.
    output['nodes_dict'][rank_event].update(
        {
            'sources': list(all_events_at_this_rank),
            'color': rank_palette,
            'sources_index': list(range(i, i+len(all_events_at_this_rank)))
        }
    )
    # Finally, increment by the length of this rank's available events to make sure next indices will not be chosen from existing ones
    i += len(output['nodes_dict'][rank_event]['sources_index'])

In [6]:
# Working on the links_dict

output.update({'links_dict': dict()})

# Group the DataFrame by user_id and rank_event
grouped = data.groupby(['user_id', 'rank_event'])

# Define a function to read the souces, targets, values and time from event to next_event:
def update_source_target(user):
    try:
        # user.name[0] is the user's user_id; user.name[1] is the rank of each action
        # 1st we retrieve the source and target's indices from nodes_dict
        source_index = output['nodes_dict'][user.name[1]]['sources_index'][output['nodes_dict']
                                                                           [user.name[1]]['sources'].index(user['event_name'].values[0])]
        target_index = output['nodes_dict'][user.name[1] + 1]['sources_index'][output['nodes_dict']
                                                                               [user.name[1] + 1]['sources'].index(user['next_event'].values[0])]

         # If this source is already in links_dict...
        if source_index in output['links_dict']:
            # ...and if this target is already associated to this source...
            if target_index in output['links_dict'][source_index]:
                # ...then we increment the count of users with this source/target pair by 1, and keep track of the time from source to target
                output['links_dict'][source_index][target_index]['unique_users'] += 1
                output['links_dict'][source_index][target_index]['avg_time_to_next'] += user['time_to_next'].values[0]
            # ...but if the target is not already associated to this source...
            else:
                # ...we create a new key for this target, for this source, and initiate it with 1 user and the time from source to target
                output['links_dict'][source_index].update({target_index:
                                                           dict(
                                                               {'unique_users': 1,
                                                                'avg_time_to_next': user['time_to_next'].values[0]}
                                                           )
                                                           })
        # ...but if this source isn't already available in the links_dict, we create its key and the key of this source's target, and we initiate it with 1 user and the time from source to target
        else:
            output['links_dict'].update({source_index: dict({target_index: dict(
                {'unique_users': 1, 'avg_time_to_next': user['time_to_next'].values[0]})})})
    except Exception as e:
        pass

# Apply the function to your grouped Pandas object:
grouped.apply(lambda user: update_source_target(user)) 

""


In [7]:
targets = []
sources = []
values = []
time_to_next = []

for source_key, source_value in output['links_dict'].items():
    for target_key, target_value in output['links_dict'][source_key].items():
        sources.append(source_key)
        targets.append(target_key)
        values.append(target_value['unique_users'])
        time_to_next.append(str(pd.to_timedelta(
            target_value['avg_time_to_next'] / target_value['unique_users'])).split('.')[0]) # Split to remove the milliseconds information

labels = []
colors = []
for key, value in output['nodes_dict'].items():
    labels = labels + list(output['nodes_dict'][key]['sources'])
    colors = colors + list(output['nodes_dict'][key]['color'])

for idx, color in enumerate(colors):
    colors[idx] = "rgb" + str(color) + ""

In [11]:
import plotly.graph_objects as go
#import chart_studio.plotly as py
import plotly

fig = go.Figure(data=[go.Sankey(
    node=dict(
        thickness=10,  # default is 20
        line=dict(color="black", width=0.5),
        label=labels,
        color=colors
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        label=time_to_next,
        hovertemplate='%{value} unique users went from %{source.label} to %{target.label}.<br />' +
        '<br />It took them %{label} in average.<extra></extra>',
    ))])

fig.update_layout(autosize=True, title_text="Medium app", font=dict(size=15), plot_bgcolor='white')

# fig.show
#publish_to_web = True
#if publish_to_web:
#   py.iplot(fig, filename='user_journey')
#else:
fig.show()